In [1]:
%run init.ipynb

/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8788 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36701 instead
  f"Port {expected} is already in use.\n"


In [2]:
import dask
import dask.dataframe as dd

In [3]:
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"

In [4]:
ddf = dd.read_parquet(f"{log_dir}/*.parquet")

ddf

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
npartitions=160,,,,,,,,,,,,,,,,,,,
,int32,int32,int32,int32,float32,float32,object,int32,int32,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [58]:
ddf.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
0,1,0,322192,0,2.823260,2.823267,__xstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
1,2,0,322192,0,2.823293,2.823305,open,0,2,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,2,,,,,,,,
2,3,0,322192,0,2.823371,2.823373,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
3,4,0,322192,0,2.823390,2.823399,read,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,16777216,,,,,,,
4,5,0,322192,0,2.823490,2.823494,close,0,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,,,,,,,,,


In [6]:
ddf.index.count().compute()

27463

In [7]:
ddf_000001 = ddf[ddf['args_1'].str.contains("000001") | ddf['args_2'].str.contains("000001")].compute()

ddf_000001

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
59,60,0,322192,0,5.638446,5.638453,__xstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
60,61,0,322192,0,5.638469,5.638579,open,0,2,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,2,,,,,,,,
61,62,0,322192,0,5.638655,5.638657,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
62,63,0,322192,0,5.773078,5.776894,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,16777216,,,,,,,
63,64,0,322192,0,5.844885,5.846870,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,16777216,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,87,1240,322192,0,344.429321,344.429321,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
87,88,1240,322192,0,344.489044,344.489075,close,0,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,,,,,,,,,
88,89,1240,322192,0,344.489075,344.489075,close,0,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,,,,,,,,,
89,90,1240,322192,0,344.489075,344.489075,close,0,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,,,,,,,,,


In [8]:
args1 = list(ddf_000001['args_1'].unique())
args2 = list(ddf_000001['args_2'].unique())
args3 = list(ddf_000001['args_3'].unique())

args1

['1',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_s.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_u.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_v.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_w.dat',
 'cm1out_diag_000001_s.dat',
 'cm1out_diag_000001_w.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_diag_000001_s.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_diag_000001_w.dat',
 'cm1rst_000001_x.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1rst_000001_s.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1rst_000001_u.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1rst_000001_v.dat',
 '/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1r

In [17]:
filename = 'cm1out_000001_s.dat'
ddf[(ddf['args_1'].str.contains(filename)) | (ddf['args_2'].str.contains(filename))].compute()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,args_2,args_3,args_4,args_5,args_6,args_7,args_8,args_9,args_10
59,60,0,322192,0,5.638446,5.638453,__xstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
60,61,0,322192,0,5.638469,5.638579,open,0,2,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,2,,,,,,,,
61,62,0,322192,0,5.638655,5.638657,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
62,63,0,322192,0,5.773078,5.776894,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,16777216,,,,,,,
63,64,0,322192,0,5.844885,5.846870,write,0,3,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,16777216,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,7,1240,322192,0,3.437691,3.437695,__xstat,0,3,1,cm1out_000001_s.dat,%p,,,,,,,
7,8,1240,322192,0,3.437723,5.588772,open,0,3,cm1out_000001_s.dat,66,438,,,,,,,
8,9,1240,322192,0,5.628821,5.628825,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,
9,10,1240,322192,0,5.628830,5.628833,__fxstat,0,3,1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,%p,,,,,,,


In [15]:
log_dir2 = "/p/gpfs1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"

ddf2 = dd.read_parquet(f"{log_dir2}/*.parquet")

In [ ]:
ddf2[(ddf2['filename'].str.contains(filename))].compute()

In [17]:
ddf2[ddf2['filename'] == ""].compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id


In [35]:
ddf2['proc_id'].nunique().compute()

1280

In [29]:
ddf2['file_id'].unique().compute()

0      4531734244992817467
1      4531734247452668476
2      4531734246070024789
3      4531734245959907731
4      4531734245058167199
              ...         
770    4531734246187791667
771    4531734246508946226
772    4531734246070374525
773    4531734245825639992
774     901974072163427748
Name: file_id, Length: 775, dtype: int64

In [60]:
ddf2['func_id'].unique().compute()

0      __xstat
1         open
2     __fxstat
3         read
4        close
5        lseek
6    ftruncate
7        write
Name: func_id, dtype: object

In [71]:
ddf2[ddf2['func_id'] == 'lseek']['file_id'].unique().compute()

0     4531734247452668476
1     4531734246070024789
2     4531734245058167199
3     4531734246809951295
4     4531734247914523647
5     4531734246793868961
6     4531734248720417485
7     4531734245717475248
8     4531734247130387716
9     4531734247883313948
10    4531734246784543057
11    4531734245586618195
12    4531734246721693969
13    4531734244455765367
Name: file_id, dtype: int64

In [30]:
ddf2['filename'].unique().compute()

0      /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
1      /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
2      /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
3      /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
4      /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
                             ...                        
770    /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
771    /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
772    /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
773    /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...
774                                            /dev/null
Name: filename, Length: 775, dtype: object

In [5]:
ddf2[ddf2['file_id'] == 4531734244992817467].compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,322192,0,3,2.823260,2.823267,__xstat,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000007,28232634,4531734244992817467,5117778030848174740
1,2,1,0,322192,0,3,2.823293,2.823305,open,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000012,28232988,4531734244992817467,5117778030848174740
2,3,1,0,322192,0,3,2.823371,2.823373,__fxstat,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000002,28233717,4531734244992817467,5117778030848174740
3,4,1,0,322192,0,1,2.823390,2.823399,read,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216,1758241.75,0.000009,28233943,4531734244992817467,5117778030848174740
4,5,1,0,322192,0,3,2.823490,2.823494,close,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000004,28234916,4531734244992817467,5117778030848174740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6285,6286,1,1279,322192,0,3,2.797101,2.797108,__xstat,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000007,27971043,4531734244992817467,5117778030931995284
6286,6287,1,1279,322192,0,3,2.797126,2.797140,open,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000014,27971328,4531734244992817467,5117778030931995284
6287,6288,1,1279,322192,0,3,2.797208,2.797210,__fxstat,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,0,0.00,0.000002,27972087,4531734244992817467,5117778030931995284
6288,6289,1,1279,322192,0,1,2.797228,2.797238,read,0,localhost,app1,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,16777216,1600000.00,0.000010,27972326,4531734244992817467,5117778030931995284


In [122]:
import itertools as it

def chunk(s):
    '''
    The function applied to the
    individual partition (map)
    '''    
    return s.apply(lambda x: list(set(x)))


def agg(s):
    '''
    The function whic will aggrgate 
    the result from all the partitions(reduce)
    '''
    s = s._selected_obj    
    return s.groupby(level=list(range(s.index.nlevels))).sum()


def finalize(s):
    '''
    The optional functional that will be 
    applied to the result of the agg_tu functions
    '''
    return s.apply(lambda x: len(set(x)))

tunique = dd.Aggregation('tunique', chunk, agg,finalize)


In [180]:
##
#   4531734247452668476
# 1     4531734246070024789
# 2     4531734245058167199
# 3     4531734246809951295
# 4     4531734247914523647
# 5     4531734246793868961
# 6     4531734248720417485
# 7     4531734245717475248
# 8     4531734247130387716
# 9     4531734247883313948
# 10    4531734246784543057
# 11    4531734245586618195
# 12    4531734246721693969
# 13    4531734244455765367
file_acc_ids = [
    4531734247452668476,
    4531734246070024789,
    4531734245058167199,
    4531734246809951295,
    4531734247914523647,
    4531734246793868961,
    4531734248720417485,
    4531734245717475248,
    4531734247130387716,
    4531734247883313948,
    4531734246784543057,
    4531734245586618195,
    4531734246721693969,
    4531734244455765367,
    
]

file_acc_stat_ddf = ddf2[ddf2['file_id'].isin(file_acc_ids)].groupby(['file_id','filename','func_id']).agg({'index':['count',min,max],'size':[min,max],'duration':sum}).compute()
file_acc_stat_ddf['duration', 'per'] = file_acc_stat_ddf.div(file_acc_stat_ddf.groupby(level=0).sum(), level=0)['duration', 'sum']

file_acc_stat_ddf[:40]

index  \
                                                                                 count   
file_id             filename                                           func_id           
4531734244455765367 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat      2   
                                                                       __xstat       1   
                                                                       close         1   
                                                                       ftruncate     1   
                                                                       lseek         1   
                                                                       open          1   
                                                                       write         1   
4531734245058167199 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat    362   
                                                                       __xstat     361   
                                                                       close       361   
                                                                       ftruncate   361   
                                                                       lseek       361   
                                                                       open        361   
                                                                       write       361   
4531734245586618195 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat      8   
                                                                       __xstat       7   
                                                                       close         7   
                                                                       lseek        12   
                                                                       open          7   
                                                                       write         7   
4531734245717475248 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat      8   
                                                                       __xstat       7   
                                                                       close         7   
                                                                       ftruncate     7   
                                                                       lseek         7   
                                                                       open          7   
                                                                       write         7   
4531734246070024789 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat      2   
                                                                       __xstat       1   
                                                                       close         1   
                                                                       ftruncate     1   
                                                                       lseek         1   
                                                                       open          1   
                                                                       write         1   
4531734246721693969 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat      2   
                                                                       __xstat       1   
                                                                       close         1   
                                                                       ftruncate     1   
                                                                       lseek         1   
                                                                       open          1   

                                                                                         \
                                                                                    min   
file_id             filename

In [181]:
print(file_acc_stat_ddf.groupby(level=0).sum())
file_acc_stat_ddf.div(file_acc_stat_ddf.groupby(level=0).sum(), level=0)[:20]

                    index                    size          duration     
                    count     min     max     min     max       sum  per
file_id                                                                 
4531734244455765367     8  105065  105066    6112    6112  0.000537  1.0
4531734245058167199  2528     249  104237    5530    5530  2.220052  1.0
4531734245586618195    48    2766   89430      20     140  0.000941  1.0
4531734245717475248    50     802  104776    3491    3491  0.001764  1.0
4531734246070024789     8     158     159    2055    2055  0.002457  1.0
4531734246721693969     8   52978   52979    6112    6112  0.000904  1.0
4531734246784543057  2526     369   89388     272   98192  0.027824  1.0
4531734246793868961    50     690  104678    2602    2602  0.001811  1.0
4531734246809951295    50     389  104384     643     643  0.001883  1.0
4531734247130387716  2528     942  104909   14184   14184  0.147503  1.0
4531734247452668476     8      67      68  122854  

index  \
                                                                                     count   
file_id             filename                                           func_id               
4531734244455765367 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.250000   
                                                                       __xstat    0.125000   
                                                                       close      0.125000   
                                                                       ftruncate  0.125000   
                                                                       lseek      0.125000   
                                                                       open       0.125000   
                                                                       write      0.125000   
4531734245058167199 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.143196   
                                                                       __xstat    0.142801   
                                                                       close      0.142801   
                                                                       ftruncate  0.142801   
                                                                       lseek      0.142801   
                                                                       open       0.142801   
                                                                       write      0.142801   
4531734245586618195 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.166667   
                                                                       __xstat    0.145833   
                                                                       close      0.145833   
                                                                       lseek      0.250000   
                                                                       open       0.145833   
                                                                       write      0.145833   

                                                                                            \
                                                                                       min   
file_id             filename                                           func_id               
4531734244455765367 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.142769   
                                                                       __xstat    0.142750   
                                                                       close      0.142959   
                                                                       ftruncate  0.142912   
                                                                       lseek      0.142902   
                                                                       open       0.142759   
                                                                       write      0.142950   
4531734245058167199 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.136546   
                                                                       __xstat    0.128514   
                                                                       close      0.156627   
                                                                       ftruncate  0.148594   
                                                                       lseek      0.144578   
                                                                       open       0.132530   
                                                                       write      0.152610   
4531734245586618195 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... __fxstat   0.017354   
                                                                       __xstat    0.016631   
                                                                       close      0.018438   
                                          

In [103]:
file_func_stat_ddf = ddf2[ddf2['file_id'].isin(file_acc_ids)].groupby(['file_id','filename','io_cat', 'func_id']).agg({
    'index': 'count',
    'duration': 'sum',
    'size': 'sum',
    'proc_id': tunique
}).compute()

file_func_stat_ddf[:40]

index  \
file_id             filename                                           io_cat func_id            
4531734244455765367 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write          1   
                                                                       3      __fxstat       2   
                                                                              __xstat        1   
                                                                              close          1   
                                                                              ftruncate      1   
                                                                              lseek          1   
                                                                              open           1   
4531734245058167199 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write        361   
                                                                       3      __fxstat     362   
                                                                              __xstat      361   
                                                                              close        361   
                                                                              ftruncate    361   
                                                                              lseek        361   
                                                                              open         361   
4531734245586618195 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write          7   
                                                                       3      __fxstat       8   
                                                                              __xstat        7   
                                                                              close          7   
                                                                              lseek         12   
                                                                              open           7   
4531734245717475248 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write          7   
                                                                       3      __fxstat       8   
                                                                              __xstat        7   
                                                                              close          7   
                                                                              ftruncate      7   
                                                                              lseek          7   
                                                                              open           7   
4531734246070024789 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write          1   
                                                                       3      __fxstat       2   
                                                                              __xstat        1   
                                                                              close          1   
                                                                              ftruncate      1   
                                                                              lseek          1   
                                                                              open           1   
4531734246721693969 /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_file... 2      write          1   
                                                                       3      __fxstat       2   
                                                                              __xstat        1   
                                                                              close          1   
                                                                              ftruncate      1   
                                                                       

In [78]:
file_acc_stat_ddf2 = ddf2[ddf2['file_id'] == 4531734245058167199][['index','proc_id','file_id','filename','func_id','size']].compute()

file_acc_stat_ddf2

,index,proc_id,file_id,filename,func_id,size
31,32,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,__xstat,0
32,33,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,open,0
33,34,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,__fxstat,0
34,35,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,__fxstat,0
35,36,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,lseek,0
...,...,...,...,...,...,...
14889,14890,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,__fxstat,0
14890,14891,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,lseek,0
14891,14892,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,ftruncate,0
14892,14893,5117778030848174740,4531734245058167199,/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_fil...,write,5530


In [40]:
proc_func_stat_ddf = ddf2[ddf2['proc_id'] == 5117778030848174740].groupby(['proc_id', 'io_cat', 'func_id']).agg({
    'index': 'count',
    'duration': 'sum',
    'size': 'sum',
    'file_id': tunique
}).compute()

proc_func_stat_ddf

index  duration        size  file_id
proc_id             io_cat func_id                                        
5117778030848174740 1      read           3  0.001326    50331648        2
                    2      write       2303  0.579398  1230852944      772
                    3      __fxstat    2993  0.005668           0      774
                           __xstat     2251  0.016423           0      774
                           close       2251  0.069000           0      774
                           ftruncate   1122  0.163601           0       12
                           lseek       1854  0.004011    17674980       14
                           open        2251  3.275673           0      774

In [ ]:
ddf2[ddf2['func_id'].str.contains('seek') & ddf2['size'] > 0].compute()

In [5]:
log_dir_h = "/p/gpfs1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
# /p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4
ddf_h = dd.read_parquet(f"{log_dir_h}/*.parquet")

ddf_h

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
npartitions=1,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
ddf_h.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,312784,2,0,0.968866,0.968892,MPI_Comm_size,0,localhost,app1,,0,0.0,0.000026,9688790,6142509188972423790,5117778030848165332
1,2,1,0,312784,2,0,0.968923,0.968924,MPI_Comm_rank,0,localhost,app1,,0,0.0,0.000001,9689235,6142509188972423790,5117778030848165332
2,3,1,0,312784,4,0,1.127011,1.127014,_ZN15RestartIO_GLEANC1Ev,0,localhost,app1,,0,0.0,0.000003,11270123,6142509188972423790,5117778030848165332
3,4,1,0,312784,2,0,1.127036,4.299232,MPI_Comm_dup,0,localhost,app1,,0,0.0,3.172197,27131343,6142509188972423790,5117778030848165332
4,5,1,0,312784,0,3,4.298360,4.298391,open,1,localhost,app1,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0.0,0.000031,42983753,3341432783455051907,5117778030848165332


In [7]:
ddf_h['file_id'].nunique().compute()

2562

In [93]:
ddf_h['func_id'].unique().compute()

0                                         MPI_Comm_size
1                                         MPI_Comm_rank
2                              _ZN15RestartIO_GLEANC1Ev
3                                          MPI_Comm_dup
4                                                  open
5                                             ftruncate
6                                                 close
7                                                unlink
8     _ZN15RestartIO_GLEAN23__duplicateCommunicatorE...
9      _ZN15RestartIO_GLEAN24__initalizePartitionInfoEv
10                                       MPI_Comm_split
11           _ZN15RestartIO_GLEAN18__createPartitionsEv
12    _ZN15RestartIO_GLEAN10InitializeEP19ompi_commu...
13        _ZN15RestartIO_GLEAN21SetPOSIX_IO_InterfaceEi
14                                        MPI_Allreduce
15                                           MPI_Gather
16                                               open64
17                                          MPI_

In [8]:
file_acc_ids_h = list(ddf_h[ddf_h['func_id'] == 'lseek64']['file_id'].unique().compute())

file_acc_ids_h[:10]

[5347710266301943549,
 5347710268972954261,
 5347710268382397342,
 5347710264855252483,
 5347710268895530979,
 5347710265310567791,
 5347710265562864681,
 5347710267155037871,
 5347710267821916138,
 5347710267671485346]

In [9]:
file_acc_stat_ddf_h = ddf_h[ddf_h['file_id'].isin(file_acc_ids_h)].groupby(['file_id','filename','func_id']).agg({'index':['count',min,max],'size':[min,max],'duration':sum}).compute()
file_acc_stat_ddf_h['duration', 'per'] = file_acc_stat_ddf_h.div(file_acc_stat_ddf_h.groupby(level=0).sum(), level=0)['duration', 'sum']

file_acc_stat_ddf_h[:40]

index  \
                                                                               count   
file_id             filename                                           func_id         
5347710264681493468 /p/gpfs1/iopp/temp/hacc_dir//test-Part00001027-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001027-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264689155874 /p/gpfs1/iopp/temp/hacc_dir//test-Part00000350-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00000350-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264696876807 /p/gpfs1/iopp/temp/hacc_dir//test-Part00000147-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00000147-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264698166716 /p/gpfs1/iopp/temp/hacc_dir//test-Part00000494-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00000494-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264703891466 /p/gpfs1/iopp/temp/hacc_dir//test-Part00001029-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001029-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264705602614 /p/gpfs1/iopp/temp/hacc_dir//test-Part00000336-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00000336-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   
                                                                       read       10   
                                                                       write      10   
5347710264706452906 /p/gpfs1/iopp/temp/hacc_dir//test-Part00001220-... open64      1   
                    /p/gpfs1/iopp/temp/hacc_dir/test-Part00001220-o... close       4   
                                                                       lseek64    20   
                                                                       open64      3   

                                                                                        \
                                                                                   min   
file_id             filename                                           func_id           
5347710264681493468 /p/g

In [49]:
ddf_h[ddf_h['func_id'] == 'MPI_Allreduce'].compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
23,24,1,0,312784,2,0,10.648633,10.648634,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000001,106486336,6142509188972423790,5117778030848165332
24,25,1,0,312784,2,0,10.648639,10.648885,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000246,106487621,6142509188972423790,5117778030848165332
209,210,1,1,312784,2,0,10.662471,10.662474,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000003,106624719,6142509188972423790,5117778030848230868
210,211,1,1,312784,2,0,10.662478,10.662730,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000252,106626044,6142509188972423790,5117778030848230868
336,337,1,2,312784,2,0,10.659667,10.659669,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000002,106596678,6142509188972423790,5117778030848296404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162230,162231,1,1277,312784,2,0,4.657865,4.658106,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000241,46579856,6142509188972423790,5117778030931854804
162356,162357,1,1278,312784,2,0,4.722832,4.722835,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000003,47228331,6142509188972423790,5117778030931920340
162357,162358,1,1278,312784,2,0,4.722840,4.723229,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000389,47230342,6142509188972423790,5117778030931920340
162483,162484,1,1279,312784,2,0,4.700896,4.700898,MPI_Allreduce,0,localhost,app1,,0,0.0,0.000002,47008968,6142509188972423790,5117778030931985876


In [50]:
ddf_h[ddf_h['file_id'] == 5347710266789296543].compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
10498,10499,1,82,312784,0,3,4.720165,13.159923,open64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir//test-Part00000082...,0,0.000000e+00,8.439758,89400437,5347710266789296543,5117778030853539284
10499,10500,1,82,312784,0,3,13.497088,13.497097,close,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,0,0.000000e+00,0.000008,134970927,5347710266789296543,5117778030853539284
10501,10502,1,82,312784,0,3,13.525151,13.525364,open64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,0,0.000000e+00,0.000212,135252576,5347710266789296543,5117778030853539284
10505,10506,1,82,312784,0,3,22.427418,22.427420,lseek64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,25165824,1.090909e+07,0.000002,224274195,5347710266789296543,5117778030853539284
10506,10507,1,82,312784,0,2,22.427425,22.641762,write,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,67108864,2.985953e+02,0.214337,225345929,5347710266789296543,5117778030853539284
10508,10509,1,82,312784,0,3,22.641792,22.641796,lseek64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,92274688,3.142857e+07,0.000003,226417940,5347710266789296543,5117778030853539284
10509,10510,1,82,312784,0,2,22.641798,22.653433,write,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,67108864,5.501165e+03,0.011634,226476154,5347710266789296543,5117778030853539284
10511,10512,1,82,312784,0,3,22.653490,22.653496,lseek64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,159383552,2.533333e+07,0.000006,226534925,5347710266789296543,5117778030853539284
10512,10513,1,82,312784,0,2,22.653498,23.002195,write,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,67108864,1.835398e+02,0.348698,228278464,5347710266789296543,5117778030853539284
10514,10515,1,82,312784,0,3,23.002258,23.002272,lseek64,0,localhost,app1,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000082-...,226492416,1.648855e+07,0.000013,230022648,5347710266789296543,5117778030853539284


In [6]:
log_dir_m = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
# /p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4
ddf_m = dd.read_parquet(f"{log_dir_m}/*.parquet")

ddf_m

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
npartitions=10,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
ddf_m.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,bandwidth,duration,tmid,file_id,proc_id
0,1,61856,0,314672,0,3,2.798459,2.798468,__xstat64,0,lassen70,mImgtbl,/p/gpfs1/iopp/temp/montage/2854964/0,0,0.0,0.000009,27984635,1571755609790269981,244022999328542004
1,2,61856,0,314672,0,3,2.798558,2.798704,fopen64,0,lassen70,mImgtbl,/p/gpfs1/iopp/temp/montage/2854964/0/run/Kimag...,0,0.0,0.000146,27986308,7672382664121083431,244022999328542004
2,3,61856,0,314672,0,3,2.798825,2.798844,opendir,0,lassen70,mImgtbl,/p/gpfs1/iopp/temp/montage/2854964/0,0,0.0,0.000019,27988344,1571755609790269981,244022999328542004
3,4,61856,0,314672,0,3,2.798926,2.798935,__xstat64,0,lassen70,mImgtbl,/p/gpfs1/iopp/temp/montage/2854964/0,0,0.0,0.000009,27989303,1571755609790269981,244022999328542004
4,5,61856,0,314672,0,3,2.798986,2.798995,__xstat64,0,lassen70,mImgtbl,/p/gpfs1/iopp/temp/montage/2854964,0,0.0,0.000009,27989902,5347462497940582382,244022999328542004


In [8]:
ddf_m['file_id'].nunique().compute()

1924

In [9]:
ddf_m['proc_id'].nunique().compute()

255

In [10]:
ddf_m['index'].count().compute()

4997828